## PART 2 Web scrapping of Population and Demographics data of New York city from Wikipedia ##

In [2]:
import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium 
import csv 

import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2020.4.5.2 |       hecda079_0         147 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2020.4.5.2         |   py36h9f0ad1d_0         152 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0           conda-forge
    geopy:          

In [3]:
website_url = requests.get('https://en.wikipedia.org/wiki/Demographics_of_New_York_City').text
soup = BeautifulSoup(website_url,'lxml')
table = soup.find('table',{'class':'wikitable sortable'})
#print(soup.prettify())

headers = [header.text for header in table.find_all('th')]

table_rows = table.find_all('tr')        
rows = []
for row in table_rows:
   td = row.find_all('td')
   row = [row.text for row in td]
   rows.append(row)

with open('BON2_POPULATION1.csv', 'w') as f:
   writer = csv.writer(f)
   writer.writerow(headers)
   writer.writerows(row for row in rows if row)

In [4]:
Pop_data=pd.read_csv('BON2_POPULATION1.csv')
Pop_data.drop(Pop_data.columns[[7,8,9,10,11]], axis=1,inplace=True)
print('Data downloaded!')

Data downloaded!


In [5]:
Pop_data.columns = Pop_data.columns.str.replace(' ', '')
Pop_data.columns = Pop_data.columns.str.replace('\'','')
Pop_data.rename(columns={'Borough':'persons_sq_mi','County':'persons_sq_km'}, inplace=True)
Pop_data

,NewYorkCitysfiveboroughsvte,Jurisdiction,Population,GrossDomesticProduct,Landarea,Density,persons_sq_mi,squarekm,persons/sq.mi,persons/km2
0,The Bronx\n,\n Bronx\n,"1,418,207\n",42.695\n,"30,100\n",42.10\n,109.04\n,NaN,NaN,NaN
1,Brooklyn\n,\n Kings\n,"2,559,903\n",91.559\n,"35,800\n",70.82\n,183.42\n,NaN,NaN,NaN
2,Manhattan\n,\n New York\n,"1,628,706\n",600.244\n,"368,500\n",22.83\n,59.13\n,NaN,NaN,NaN
3,Queens\n,\n Queens\n,"2,253,858\n",93.310\n,"41,400\n",108.53\n,281.09\n,NaN,NaN,NaN
4,Staten Island\n,\n Richmond\n,"476,143\n",14.514\n,"30,500\n",58.37\n,151.18\n,NaN,NaN,NaN
5,City of New York,"8,336,817",842.343,"101,000",302.64,783.83,"27,547",NaN,NaN,NaN
6,State of New York,"19,453,561","1,731.910","89,000","47,214","122,284",412,NaN,NaN,NaN
7,Sources:[14] and see individual borough articl...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
Pop_data.rename(columns = {'NewYorkCitysfiveboroughsvte\n' : 'Borough',
                   'Jurisdiction\n':'County',
                   'Population\n':'Estimate_2017', 
                   'Landarea\n':'square_miles',
                    'Density\n':'square_km'}, inplace=True)
Pop_data

,Borough,County,Estimate_2017,GrossDomesticProduct,square_miles,square_km,persons_sq_mi,squarekm,persons/sq.mi,persons/km2
0,The Bronx\n,\n Bronx\n,"1,418,207\n",42.695\n,"30,100\n",42.10\n,109.04\n,NaN,NaN,NaN
1,Brooklyn\n,\n Kings\n,"2,559,903\n",91.559\n,"35,800\n",70.82\n,183.42\n,NaN,NaN,NaN
2,Manhattan\n,\n New York\n,"1,628,706\n",600.244\n,"368,500\n",22.83\n,59.13\n,NaN,NaN,NaN
3,Queens\n,\n Queens\n,"2,253,858\n",93.310\n,"41,400\n",108.53\n,281.09\n,NaN,NaN,NaN
4,Staten Island\n,\n Richmond\n,"476,143\n",14.514\n,"30,500\n",58.37\n,151.18\n,NaN,NaN,NaN
5,City of New York,"8,336,817",842.343,"101,000",302.64,783.83,"27,547",NaN,NaN,NaN
6,State of New York,"19,453,561","1,731.910","89,000","47,214","122,284",412,NaN,NaN,NaN
7,Sources:[14] and see individual borough articl...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
Pop_data['Borough']=Pop_data['Borough'].replace(to_replace='\n', value='', regex=True)
Pop_data['County']=Pop_data['County'].replace(to_replace='\n', value='', regex=True)
Pop_data['Estimate_2017']=Pop_data['Estimate_2017'].replace(to_replace='\n', value='', regex=True)
Pop_data['square_miles']=Pop_data['square_miles'].replace(to_replace='\n', value='', regex=True)
Pop_data['square_km']=Pop_data['square_km'].replace(to_replace='\n', value='', regex=True)
Pop_data['persons_sq_mi']=Pop_data['persons_sq_mi'].replace(to_replace='\n', value='', regex=True)
Pop_data

,Borough,County,Estimate_2017,GrossDomesticProduct,square_miles,square_km,persons_sq_mi,squarekm,persons/sq.mi,persons/km2
0,The Bronx,Bronx,"1,418,207",42.695\n,"30,100",42.10,109.04,NaN,NaN,NaN
1,Brooklyn,Kings,"2,559,903",91.559\n,"35,800",70.82,183.42,NaN,NaN,NaN
2,Manhattan,New York,"1,628,706",600.244\n,"368,500",22.83,59.13,NaN,NaN,NaN
3,Queens,Queens,"2,253,858",93.310\n,"41,400",108.53,281.09,NaN,NaN,NaN
4,Staten Island,Richmond,"476,143",14.514\n,"30,500",58.37,151.18,NaN,NaN,NaN
5,City of New York,"8,336,817",842.343,"101,000",302.64,783.83,"27,547",NaN,NaN,NaN
6,State of New York,"19,453,561","1,731.910","89,000","47,214","122,284",412,NaN,NaN,NaN
7,Sources:[14] and see individual borough articles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Shift data in the last two rows

In [8]:
Pop_data.loc[5:,['square_km','persons_sq_mi']] = Pop_data.loc[2:,['square_km','persons_sq_mi']].shift(1,axis=1)
Pop_data.loc[5:,['square_miles','square_km']] = Pop_data.loc[2:,['square_miles','square_km']].shift(1,axis=1)
Pop_data.loc[5:,['Estimate_2017','square_miles']] = Pop_data.loc[2:,['Estimate_2017','square_miles']].shift(1,axis=1)
Pop_data.loc[5:,['County','Estimate_2017']] = Pop_data.loc[2:,['County','Estimate_2017']].shift(1,axis=1)
Pop_data.loc[5:,['Borough','County']] = Pop_data.loc[2:,['Borough','County']].shift(1,axis=1)
Pop_data

,Borough,County,Estimate_2017,GrossDomesticProduct,square_miles,square_km,persons_sq_mi,squarekm,persons/sq.mi,persons/km2
0,The Bronx,Bronx,"1,418,207",42.695\n,"30,100",42.10,109.04,NaN,NaN,NaN
1,Brooklyn,Kings,"2,559,903",91.559\n,"35,800",70.82,183.42,NaN,NaN,NaN
2,Manhattan,New York,"1,628,706",600.244\n,"368,500",22.83,59.13,NaN,NaN,NaN
3,Queens,Queens,"2,253,858",93.310\n,"41,400",108.53,281.09,NaN,NaN,NaN
4,Staten Island,Richmond,"476,143",14.514\n,"30,500",58.37,151.18,NaN,NaN,NaN
5,NaN,City of New York,"8,336,817","101,000",842.343,302.64,783.83,NaN,NaN,NaN
6,NaN,State of New York,"19,453,561","89,000","1,731.910","47,214","122,284",NaN,NaN,NaN
7,NaN,Sources:[14] and see individual borough articles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Remove 'NAN'

In [9]:
Pop_data = Pop_data.fillna('')
Pop_data

,Borough,County,Estimate_2017,GrossDomesticProduct,square_miles,square_km,persons_sq_mi,squarekm,persons/sq.mi,persons/km2
0,The Bronx,Bronx,"1,418,207",42.695\n,"30,100",42.10,109.04,,,
1,Brooklyn,Kings,"2,559,903",91.559\n,"35,800",70.82,183.42,,,
2,Manhattan,New York,"1,628,706",600.244\n,"368,500",22.83,59.13,,,
3,Queens,Queens,"2,253,858",93.310\n,"41,400",108.53,281.09,,,
4,Staten Island,Richmond,"476,143",14.514\n,"30,500",58.37,151.18,,,
5,,City of New York,"8,336,817","101,000",842.343,302.64,783.83,,,
6,,State of New York,"19,453,561","89,000","1,731.910","47,214","122,284",,,
7,,Sources:[14] and see individual borough articles,,,,,,,,


Drop the last row

In [10]:
i = Pop_data[((Pop_data.County == 'Sources:[14] and see individual borough articles'))].index
Pop_data.drop(i)

,Borough,County,Estimate_2017,GrossDomesticProduct,square_miles,square_km,persons_sq_mi,squarekm,persons/sq.mi,persons/km2
0,The Bronx,Bronx,"1,418,207",42.695\n,"30,100",42.10,109.04,,,
1,Brooklyn,Kings,"2,559,903",91.559\n,"35,800",70.82,183.42,,,
2,Manhattan,New York,"1,628,706",600.244\n,"368,500",22.83,59.13,,,
3,Queens,Queens,"2,253,858",93.310\n,"41,400",108.53,281.09,,,
4,Staten Island,Richmond,"476,143",14.514\n,"30,500",58.37,151.18,,,
5,,City of New York,"8,336,817","101,000",842.343,302.64,783.83,,,
6,,State of New York,"19,453,561","89,000","1,731.910","47,214","122,284",,,


Save dataframe as csv file

In [11]:
Pop_data.to_csv('BON2_POPULATION.csv',index=False)

#### DEMOGRAPHICS DATA ####

We will web scrap Demographics data from wikipedia page - https://en.wikipedia.org/wiki/New_York_City
Web scrapping of Demographics data from wikipedia page using BeautifulSoup.

In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [13]:
url = 'https://en.wikipedia.org/wiki/New_York_City'
website_url = requests.get(url).text
soup = BeautifulSoup(website_url,'html.parser')
tables = soup.find_all('table')

for table in tables:
    if 'Racial composition' in table.text:
        headers = [header.text.strip() for header in table.find_all('th')]
        rows = []
        table_rows = table.find_all('tr')    
        for row in table_rows:
           td = row.find_all('td')
           row = [row.text for row in td]
           rows.append(row)

df = pd.DataFrame(rows, columns=headers) 

In [14]:
print(df)

                 Racial composition 2010[235] 1990[237]   1970[237] 1940[237]
0                              None      None      None        None      None
1                             White     44.0%     52.3%       76.6%   93.6%\n
2                     —Non-Hispanic     33.3%     43.2%  62.9%[238]   92.0%\n
3         Black or African American     25.5%     28.7%       21.1%    6.1%\n
4  Hispanic or Latino (of any race)     28.6%     24.4%  16.2%[238]    1.6%\n
5                             Asian     12.7%      7.0%        1.2%       –\n


In [23]:
df.rename(columns = {'2010[235]' : '2010',
                   '1990[237]':'1990',
                   '1970[237]':'1970', 
                   '1940[237]':'1940',
                    }, inplace=True)
df

,Racialcomposition,2010,1990,1970,1940
0,None,None,None,None,None
1,White,44.0%,52.3%,76.6%,93.6%\n
2,—Non-Hispanic,33.3%,43.2%,62.9%[238],92.0%\n
3,Black or African American,25.5%,28.7%,21.1%,6.1%\n
4,Hispanic or Latino (of any race),28.6%,24.4%,16.2%[238],1.6%\n
5,Asian,12.7%,7.0%,1.2%,–\n


In [24]:
Demo_data = df

In [25]:
Demo_data.columns

Index(['Racialcomposition', '2010', '1990', '1970', '1940'], dtype='object')

In [26]:
Demo_data.rename(columns = {'2010[224]' : '2010',
                   '1990[226]':'1990',
                   '1970[226]':'1970', 
                   '1940[226]':'1940',
                    }, inplace=True)
Demo_data

,Racialcomposition,2010,1990,1970,1940
0,None,None,None,None,None
1,White,44.0%,52.3%,76.6%,93.6%\n
2,—Non-Hispanic,33.3%,43.2%,62.9%[238],92.0%\n
3,Black or African American,25.5%,28.7%,21.1%,6.1%\n
4,Hispanic or Latino (of any race),28.6%,24.4%,16.2%[238],1.6%\n
5,Asian,12.7%,7.0%,1.2%,–\n


In [27]:
Demo_data.columns

Index(['Racialcomposition', '2010', '1990', '1970', '1940'], dtype='object')

In [28]:
Demo_data.columns = Demo_data.columns.str.replace(' ', '')

Replace newline('\n') from each string from left and right sides

In [29]:
Demo_data= Demo_data.replace('\n',' ', regex=True)
Demo_data

,Racialcomposition,2010,1990,1970,1940
0,None,None,None,None,None
1,White,44.0%,52.3%,76.6%,93.6%
2,—Non-Hispanic,33.3%,43.2%,62.9%[238],92.0%
3,Black or African American,25.5%,28.7%,21.1%,6.1%
4,Hispanic or Latino (of any race),28.6%,24.4%,16.2%[238],1.6%
5,Asian,12.7%,7.0%,1.2%,–


In [32]:
Demo_data['1970'] = Demo_data['1970'].str.rstrip('[238]')
Demo_data

,Racialcomposition,2010,1990,1970,1940
0,None,None,None,None,None
1,White,44.0%,52.3%,76.6%,93.6%
2,—Non-Hispanic,33.3%,43.2%,62.9%,92.0%
3,Black or African American,25.5%,28.7%,21.1%,6.1%
4,Hispanic or Latino (of any race),28.6%,24.4%,16.2%,1.6%
5,Asian,12.7%,7.0%,1.2%,–


In [33]:
Demo_data.to_csv('BON2_DEMOGRAPHICS.csv',index=False)